# tutorial

A. link to helpful python docs

Recapitation, Simplification, and Mutation: https://pyslim.readthedocs.io/en/latest/tutorial.html#recapitation-simplification-and-mutation

Saving Individuals in .trees file: https://pyslim.readthedocs.io/en/latest/tutorial.html#obtaining-and-saving-individuals

Some Basic Plotting:
https://www.geeksforgeeks.org/graph-plotting-in-python-set-1/

(All this info may feel overwhelming/confusing in the beginning, so feel free to stick to the tutorial and browse through some links for extra detail/help!)

B. Which packages you need and what they do and link to docs

What is pyslim + how to install: https://pypi.org/project/pyslim/

What is tskit: https://tskit.dev/tskit/docs/stable/introduction.html
    + Install tskit: https://tskit.dev/tskit/docs/stable/installation.html

What is NumPy, install, import: https://numpy.org/doc/stable/user/absolute_beginners.html

What is Msprime: https://tskit.dev/msprime/docs/stable/quickstart.html
    + Install Msprime: https://tskit.dev/msprime/docs/stable/installation.html#sec-installation

Install IPython: https://ipython.org/install.html
	(SVG from IPython.display allows us to display trees in SVG format)
    
What is Matplotlib: https://matplotlib.org/stable/index.html
    + Install Matplotlib: https://matplotlib.org/stable/users/installing.html
    (We can use things like matplotlib.pyplot as plt for plotting, matplotlib.cm as cm 
    for color maps, and various widgets)

In [3]:
#We can import these packages in our notebooks like this:
import pyslim, tskit
import numpy as np
import msprime
from IPython.display import SVG #for displaying trees in svg format
import matplotlib
import matplotlib.pyplot as plt #plotting
import matplotlib.cm as cm #color maps
%matplotlib inline

## 1. load and simplify tree sequence to a sample

include slim code? or in github

## 2. locate children and parents

## 3. 2D plotting space trees

## 4. 3D plotting

# extensions

## mutations/selected alleles

## discrete space / metapopulations

## slendr